<a href="https://colab.research.google.com/github/oskargirardin/CRP_GDPR_datasets/blob/master/CTGAN_tests_Oskar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install ctgan --quiet
!pip install table_evaluator --quiet
! pip install kaggle --quiet
! pip install seaborn --quiet
! pip install sdmetrics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 7.4 MB/s eta 0:00:00


In [2]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d subhamjain/loan-prediction-based-on-customer-behavior

!unzip loan-prediction-based-on-customer-behavior.zip

 58% 3.00M/5.15M [00:00<00:00, 5.75MB/s]
100% 5.15M/5.15M [00:00<00:00, 7.60MB/s]
Archive:  loan-prediction-based-on-customer-behavior.zip
  inflating: Sample Prediction Dataset.csv  
  inflating: Test Data.csv           
  inflating: Training Data.csv       


In [3]:
from ctgan import CTGAN
import pandas as pd

In [4]:
data = pd.read_csv("Training Data.csv")
data = data.drop("Id", axis = 1)

In [5]:
data

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,1303834,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...
251995,8154883,43,13,single,rented,no,Surgeon,Kolkata,West_Bengal,6,11,0
251996,2843572,26,10,single,rented,no,Army_officer,Rewa,Madhya_Pradesh,6,11,0
251997,4522448,46,7,single,rented,no,Design_Engineer,Kalyan-Dombivli,Maharashtra,7,12,0
251998,6507128,45,0,single,rented,no,Graphic_Designer,Pondicherry,Puducherry,0,10,0


In [6]:
discrete_columns = data.columns[1:]

In [7]:
ctgan = CTGAN(verbose=True)

ctgan.fit(data, discrete_columns, epochs=10)

Epoch 1, Loss G:  0.4629,Loss D:  0.0469
Epoch 2, Loss G: -0.4472,Loss D:  0.0235
Epoch 3, Loss G: -1.2255,Loss D: -0.0487
Epoch 4, Loss G: -1.6698,Loss D: -0.0193
Epoch 5, Loss G: -1.7664,Loss D: -0.0143
Epoch 6, Loss G: -1.8757,Loss D: -0.0700
Epoch 7, Loss G: -1.8679,Loss D:  0.0626
Epoch 8, Loss G: -1.7804,Loss D: -0.0399
Epoch 9, Loss G: -1.7098,Loss D: -0.0627
Epoch 10, Loss G: -1.7488,Loss D:  0.0410


In [12]:
sample = ctgan.sample(1000)

In [15]:
from sdmetrics import load_demo
from sdmetrics.reports.single_table import QualityReport

real_data, synthetic_data, metadata = load_demo(modality='single_table')

my_report = QualityReport()
my_report.generate(real_data, synthetic_data, metadata)

Creating report: 100%|██████████| 4/4 [00:02<00:00,  1.34it/s]



Overall Quality Score: 81.44%

Properties:
Column Shapes: 81.56%
Column Pair Trends: 81.33%


In [19]:
my_metadata = {
  'fields': 
    {
    'Income': {'type': 'numerical', 'subtype': 'integer'},
    'Age': {'type': 'numerical', 'subtype': 'integer'},
    'Experience': {'type': 'numerical', 'subtype': 'integer'},
    'CURRENT_JOB_YRS': {'type': 'numerical', 'subtype': 'integer'},
    'CURRENT_HOUSE_YRS': {'type': 'numerical', 'subtype': 'integer'},
    'Married/Single': {'type': 'categorical'},
    'House_Ownership': {'type': 'categorical'},
    'Car_Ownership': {'type': 'categorical'},
    'Profession': {'type': 'categorical'},
    'CITY': {'type': 'categorical'},
    'STATE': {'type': 'categorical'},
    'Risk_Flag': {'type': 'boolean'}
  },
  'constraints': [],
  'model_kwargs': {},
  'name': None,
  'primary_key': None,
  'sequence_index': None,
  'entity_columns': [],
  'context_columns': []
}

In [20]:
report = QualityReport()
report.generate(data, sample, my_metadata)

Creating report: 100%|██████████| 4/4 [00:06<00:00,  1.74s/it]



Overall Quality Score: 80.34%

Properties:
Column Shapes: 86.8%
Column Pair Trends: 73.88%


In [22]:
report.get_details(property_name='Column Shapes')

,Column,Metric,Quality Score
0,Income,KSComplement,0.720877
1,Age,KSComplement,0.940770
2,Experience,KSComplement,0.908135
3,CURRENT_JOB_YRS,KSComplement,0.875103
4,CURRENT_HOUSE_YRS,KSComplement,0.835004
5,Married/Single,TVComplement,0.958905
6,House_Ownership,TVComplement,0.976508
7,Car_Ownership,TVComplement,0.962587
8,Profession,TVComplement,0.805508
9,CITY,TVComplement,0.697448


In [23]:
report.get_visualization(property_name='Column Pair Trends')

In [27]:
from sdmetrics.reports.utils import get_column_plot
get_column_plot(
    real_data=data,
    synthetic_data=sample,
    metadata=my_metadata,
    column_name="Age"
)

In [28]:
get_column_plot(
    real_data=data,
    synthetic_data=sample,
    metadata=my_metadata,
    column_name="Experience"
)